In [1]:
import datetime
import os

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(os.path.dirname("logs/" + current_time + "/"), exist_ok=True)

In [2]:
import logging

logger = logging.getLogger("gan")
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler("logs/" + current_time + "/gan.log")
fh.setLevel(logging.DEBUG)
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
formatter = logging.Formatter("%(levelname)s - %(message)s")
fh.setFormatter(formatter)
sh.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(sh)
logger.debug("Starting..")

In [3]:
# Tensorboard extension
%load_ext tensorboard

logger.debug("Starting Tensorboard..")
%tensorboard --logdir logs/

Reusing TensorBoard on port 6006 (pid 21940), started 0:04:11 ago. (Use '!kill 21940' to kill it.)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import LeakyReLU
from packaging import version

logger.info("TensorFlow version: %s", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, "This notebook requires TensorFlow 2.0 or above."

INFO - TensorFlow version: 2.0.0


In [5]:
if tf.test.is_built_with_cuda():
    gpu = tf.test.is_gpu_available()
if not gpu:
    logger.debug("GPU Acceleration not available!!")

In [6]:
logger.debug("Global Variables..")
BATCH_SIZE = 32
FEAT_DIM = 100
EPOCHS = 200
logger.debug("Batch Size = %d", BATCH_SIZE)
logger.debug("Dimension of images after flattening as used for training = %d", IMG_DIM)
logger.debug("Dimension of noise input used by generator to generate fakes = %d", FEAT_DIM)
logger.debug("Number of training epochs = %d", IMG_DIM)

In [7]:
logger.debug("Using MNIST dataset..")
(X_train, _), _ = mnist.load_data()
logger.info("Number of training examples: %d", X_train.shape[0])
logger.info("Dimension of each example: %s", X_train.shape[1:] if len(X_train.shape)>1 else 1)
logger.info("Dtype of examples: %s", X_train.dtype)
logger.info("Max pixel value: %d", X_train.max())
logger.info("Min pixel value: %d", X_train.min())

INFO - Number of training examples: 60000
INFO - Dimension of each example: (28, 28)
INFO - Dtype of examples: uint8
INFO - Max pixel value: 255
INFO - Min pixel value: 0


In [8]:
logger.debug("Rescaling training values to lie between -1 and 1 (including both)..")
X_train = X_train.astype(float)
X_train = (X_train-127.5)/127.5
logger.info("Range of X_train values: %d to %d", X_train.min(), X_train.max())

logger.debug("Reshaping training data to add depth dimension in images..")
X_train = np.reshape(X_train, (*X_train.shape, 1))
logger.info("Shape of training data: %d x %d x %d x %d", X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3])

INFO - Range of X_train values: -1 to 1
INFO - Shape of training data: 60000 x 784


In [9]:
logger.debug("Mini-Batching and Shuffling dataset..")
train_ds = tf.data.Dataset.from_tensor_slices((X_train)).shuffle(10000).batch(BATCH_SIZE)

In [10]:
logger.debug("Defining the model for discriminator..")
des_in = keras.layers.Input(shape = (28, 28, 1))
des = keras.layers.Conv2D(16, (3, 3), activation = LeakyReLU(0.2), kernel_initializer = "he_normal")(des_in)
des = keras.layers.Dropout(0.3)(des)
des = keras.layers.Dense(512, activation = LeakyReLU(0.2), kernel_initializer = "he_normal")(des)
des = keras.layers.Dropout(0.3)(des)
des = keras.layers.Dense(256, activation = LeakyReLU(0.2), kernel_initializer = "he_normal")(des)
des = keras.layers.Dropout(0.3)(des)
des = keras.layers.Dense(256, activation = LeakyReLU(0.2), kernel_initializer = "he_normal")(des)
des = keras.layers.Dropout(0.3)(des)
des_out = keras.layers.Dense(1, activation = keras.activations.sigmoid, kernel_initializer = "he_normal")(des)
discriminator  = keras.Model(inputs = des_in, outputs = des_out)
logger.debug("Discriminator Summary")
discriminator.summary(print_fn = logger.debug)

In [11]:
logger.debug("Defining generator model..")
gen_in = keras.layers.Input(shape = (100, ))
gen = keras.layers.Dense(256, activation = LeakyReLU(0.2), kernel_initializer = "he_normal")(gen_in)
gen = keras.layers.Dropout(0.3)(gen)
gen = keras.layers.Dense(512, activation = LeakyReLU(0.2), kernel_initializer = "he_normal")(gen)
gen = keras.layers.Dropout(0.3)(gen)
gen = keras.layers.Dense(1024, activation = LeakyReLU(0.2), kernel_initializer = "he_normal")(gen)
gen = keras.layers.Dropout(0.3)(gen)
gen_out = keras.layers.Dense(784, activation = keras.activations.tanh, kernel_initializer = "he_normal")(gen)
generator = keras.Model(inputs = gen_in, outputs = gen_out)
logger.debug("Generator Summary")
generator.summary(print_fn = logger.debug)

INFO - Visualization of untrained generator output


In [12]:
logger.info("Visualization of untrained generator output")
noise = tf.random.normal(shape = (1, FEAT_DIM))
generated_img = generator(noise, training = False)
plt.imshow(tf.reshape(generated_img[0], [28,28,1])[:,:,0], cmap='gray')

In [13]:
logger.debug("Defining separate optimizers for discriminator and generator..")
logger.debug("Using ADAM optimizer for both generator and discriminator!")
generator_optimizer = keras.optimizers.Adam(lr = 0.0002)
discriminator_optimizer = keras.optimizers.Adam(lr = 0.0002)

logger.debug("Using BinaryCrossentropy for loss function")
cross_entropy = keras.losses.BinaryCrossentropy()

# Storing the mean loss
discriminator_loss = tf.keras.metrics.Mean(name='discriminator_loss')
generator_loss = tf.keras.metrics.Mean(name='generator_loss')

In [14]:
# def get_discriminator_loss(real_pred, fake_pred):
#     real_loss = cross_entropy(tf.ones_like(real_pred), real_pred)
#     fake_loss = cross_entropy(tf.zeros_like(fake_pred), fake_pred)
#     total_loss = real_loss + fake_loss
#     return total_loss

# def get_generator_loss(fake_pred):
#     fake_loss = cross_entropy(tf.ones_like(fake_pred), fake_pred)
#     return fake_loss

In [15]:
# # Training function for Discriminator
# @tf.function
# def train(real):
#     noise = tf.random.normal(shape = (32, 100))
#     with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
#         fake = generator(noise, training = True)
#         real_pred = discriminator(real, training = True)
# #         real_loss = cross_entropy(tf.ones_like(real_pred), real_pred)
#         fake_pred = discriminator(fake, training = True)
# #         dis_fake_loss = cross_entropy(tf.zeros_like(fake_pred), fake_pred)
# #         gen_fake_loss = cross_entropy(tf.ones_like(fake_pred), fake_pred)
# #         total_loss = real_loss + dis_fake_loss
#         dis_loss = get_discriminator_loss(real_pred, fake_pred)
#         gen_loss = get_generator_loss(fake_pred)
#     dis_gradient = dis_tape.gradient(dis_loss, discriminator.trainable_variables)
#     gen_gradient = gen_tape.gradient(gen_loss, generator.trainable_variables)
#     discriminator_optimizer.apply_gradients(zip(dis_gradient, discriminator.trainable_variables))
    
#     generator_optimizer.apply_gradients(zip(gen_gradient, generator.trainable_variables))
    
#     discriminator_loss(dis_loss)
#     generator_loss(gen_loss)

In [16]:
# Training function for Discriminator
@tf.function
def train_discriminator(real):
    noise = tf.random.normal(shape = (32, 100))
    with tf.GradientTape() as tape:
        fake = generator(noise, training = False)
        real_pred = discriminator(real, training = True)
        real_loss = cross_entropy(tf.ones_like(real_pred), real_pred)
        fake_pred = discriminator(fake, training = True)
        fake_loss = cross_entropy(tf.zeros_like(fake_pred), fake_pred)
        total_loss = real_loss + fake_loss
    gradient = tape.gradient(total_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradient, discriminator.trainable_variables))
    
    discriminator_loss(total_loss)

In [17]:
# Training function for Generator
@tf.function
def train_generator():
    noise = tf.random.normal(shape = (32, 100))
    with tf.GradientTape() as tape:
        fake = generator(noise, training = True)
        fake_pred = discriminator(fake, training = False)
        fake_loss = cross_entropy(tf.ones_like(fake_pred), fake_pred)
    gradient = tape.gradient(fake_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradient, generator.trainable_variables))
    
    generator_loss(fake_loss)

In [18]:
train_log_dir = 'logs/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

In [ ]:
logger.debug("Beginning training..")
for epoch in range(EPOCHS):
    for real in train_ds:
#         train(real)
        train_discriminator(real)
        train_generator()
    with train_summary_writer.as_default():
        tf.summary.scalar('Discriminator loss', discriminator_loss.result(), step=epoch)
        tf.summary.scalar('Generator loss', generator_loss.result(), step=epoch)
    logger.debug("Epoch: %d - Discriminator loss: %f - Generator loss: %f", epoch+1, discriminator_loss.result(), generator_loss.result())
    discriminator_loss.reset_states()
    generator_loss.reset_states()
    if epoch%20 == 0:
        fig, ax = plt.subplots(10, 10, figsize = (20, 20))
        noise = tf.random.normal(shape = (100, 100))
        generated_images = generator(noise, training = False)
        generated_images = tf.reshape(generated_images, [100, 28, 28, 1])
        for i in range(100):
            ax[i//10, i%10].imshow(generated_images[i][:, :, 0], cmap = "gray")
        plt.savefig("progress-"+str(epoch)+".png")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
fig, ax = plt.subplots(10, 10, figsize = (20, 20))
noise = tf.random.normal(shape = (100, 100))
generated_images = generator(noise, training = False)
generated_images = tf.reshape(generated_images, [100, 28, 28, 1])
for i in range(100):
    ax[i//10, i%10].imshow(generated_images[i][:, :, 0], cmap = "gray")